In [ ]:
n_jo = 8

da = "/Users/kate/craft/data"

to = "/Users/kate/craft/workflow/Fastq.jl/tool"

be = joinpath(da, "benchmark")

gr = joinpath(da, "grch")

vb = joinpath(be, "HG002_truth/HG002_GRCh38_1_22_v4.2.1_benchmark.vcf.gz")

vc = joinpath(be, "apply_germline_dna_to_genome/call_germline_variant/pass.vcf.gz")

re = joinpath(gr, "GCA_000001405.15_GRCh38_no_alt_plus_hs38d1_analysis_set.fna")

sd = replace(re, "fna" => "sdf")

rt = "/Users/kate/craft/workflow/Fastq.jl/tool/rtg-tools-3.11/rtg"

In [ ]:
# Make vcfeval SDF

if !isdir(sd)

    run(`$rt format -o $sd $re`)

end

In [ ]:
# Rename chromosomes from numbers to strings

vcn = replace(vc, "pass" => "pass_rename_chromosomes")

if !isfile(vcn)

    ch = joinpath(gr, "chromosome/n_chrn.tsv")

    run(`bcftools annotate --threads=$n_jo --rename-chrs=$ch --output=$vcn $vc`)

end

In [ ]:
## run vcfeval

bd = joinpath(be, "HG002_truth/HG002_GRCh38_1_22_v4.2.1_benchmark_noinconsistent.bed.gz")

ouv = joinpath(be, "vcfeval")

run(`$rt vcfeval 
    --baseline=$vb 
    --bed-regions=$bd 
    --calls=$vcn 
    --template=$sd 
    --output=$ouv
    --threads=$n_jo`)

`--baseline` VCF containing baseline truth variants
`--calls` VCF containing called variants
`--template` SDF of reference genome the the `calls` VCF was made against

In [ ]:
## run hap.py in hap.py docker container

ouh = joinpath(be, "happy/")

if !isdir(ouh)

    mkdir(ouh)

end

vto = joinpath("/home", basename(to))

vda = joinpath("/home", basename(da))

vvb = joinpath(vda, split(vb, "data/")[2])

vvc = joinpath(vda, split(vcn, "data/")[2])

vbd = joinpath(vda, split(bd, "data/")[2])

vouh = joinpath(vda, split(ouh, "data/")[2], "hap.py")

vrt = joinpath(vto, basename(dirname(rt)))

vsd = joinpath(vda, split(sd, "data/")[2])

vre = joinpath(vda, split(re, "data/")[2])


println(vto)
println(vda)
println(vvb)
println(vvc)
println(vbd)
println(vouh)
println(vrt)
println(vsd)
println(vre)

id = readlines(pipeline(`docker run 
        --interactive 
        --detach 
        --tty 
        --user root 
        -v $to:$vto 
        -v $da:$vda
        pkrusche/hap.py
        bash`))

println(id)

readlines(pipeline(`docker exec --interactive $id bash -c 
                   "/opt/hap.py/bin/hap.py $vvb $vvc 
                   -f $vbd 
                   -r $vre 
                   -o $vouh 
                   --engine-vcfeval-path $vrt 
                   --engine-vcfeval-template $vsd"`))

# this command worked on 
# ./hap.py /home/data/benchmark/HG002_truth/HG002_GRCh38_1_22_v4.2.1_benchmark.vcf.gz /home/data/benchmark/apply_germline_dna_to_genome/call_germline_variant/pass_rename_chromosomes.vcf.gz -f /home/data/benchmark/HG002_truth/HG002_GRCh38_1_22_v4.2.1_benchmark_noinconsistent.bed.gz -r /home/data/grch/GCA_000001405.15_GRCh38_no_alt_plus_hs38d1_analysis_set.fna -o /home/data/benchmark/hap.py --engine-vcfeval-path /home/tool/rtg-tools-3.11 --engine-vcfeval-template /home/data/grch/GCA_000001405.15_GRCh38_no_alt_plus_hs38d1_analysis_set.sdf

#readlines(
#    pipeline(
#        `docker exec --interactive $id bash -c "/opt/hap.py/bin/hap.py $vvb $vvc -f $vbd -r $vre -o $vou -V --engine-vcfeval-path $vrt --engine-vcfeval-template $vsd"`,
#    ),
#)

# && /opt/hap.py/hap.py $vb $vcn -f $bd -o $vhao -V --verbose --engine-vcfeval-path $vrt --engine-vcfeval-template $sd"